# Introduction to Network Programming
---

## Identify a Process on Internet
A process on the Internet is identified by (*host*, *port*)
- Host(machine) by 32-bit IP address
  - dotted decimal notation: `'203.253.70.32'`
  - domain name: `'mclab.hufs.ac.kr'`
  - `'localhost' == '127.0.0.1'`
- Process in the host by 16-bit port number
  - well-known port: 0 ~ 1023
    - reserved by standard protocols
    - entifies the standard service(server process)
  - registered port: 1024 ~ 49151
  - dynamic port: 49152 ~ 65535 (dynamically assigned for clients)

## Client-Server Model
Internet에서 대부분의 application이나 application protocol은 client-server model로 구현되어 있다. 이 모델은 역할/기능이 분리되어 있어 구현이 용이하다. (Peer-to-peer model도 있다.)

Client: server에게 서비스를 request한다. (서비스를 제공하지 않는다.)

Server: client들의 request를 받아 response를 돌려 준다.
- always on: 보통 항상 실행되고 있다. - daemon process로 
- 고정 IP가 필요 (client가 server가 설치된 host를 idtify할 수 있도록)
- 여러 client들의 request들을 동시에 받고 처리할 수 있어야 한다.

Application protocol: 표준 프로토콜(예를 들어, HTTP)을 사용할 수도 있고, 사설로 새로 정의할 수도 있다. 어느쪽이든 다음의 transport protocol 위에서 구현된다.

## Transport Protocol
- TCP: 신뢰성있는 전송. connection-oriented. Byte stream으로 전달(패킷 경계가 없다.)
- UDP: 신뢰성 없음(전달하는 메시지가 손실 가능). conntectionless. 패킷으로 전달(패킷 경계가 없다.)

> Byte를 전달할 뿐, 안에 있는 내용이 문자인지, binary인지 구분하지 않는다.

## Socket API
- 가장 널리 쓰이는 Communication API(Application Programming Interface)
- 여러 protocol suite에 대해 generic API 제공
- 보통 application에서 OS 내부에 존재하는 Transport protocol간에 API를 제공
- Application에서 network layer, link layer의 서비스를 직접 이용할 수도 있다. (Super user 권한 필요)

## Socket API로 통신하려면 문자열(str)이 아니라 byte 열로 통신해야 한다.
### `str`
문자열이며 문자는 unicode라는 유일한 code point(값)을 갖는다. Unicode는 전 세계의 모든 문자를 컴퓨터에서 일관되게 표현하고 다룰 수 있도록 설계된 산업 표준이다. 

### Encoding
Unicode는 UTF-8, UTF-16 등 다양하게 encoding될 수 있다.

**UTF-8 encoding**: 문자열로 구성된 메시지를 socket API로 통신하려면 UTF-8으로 unicode를 변환해야 한다. 왜냐하면, unicode 문자는 멀티 바이트여서 중간에 잘릴 수 있다. 더 중요한 것은 HTTP에서 보는 바와 같이 많은 표준 application protocol들은 ASCII 영문자로 명령 등의 protocol 메시지를 정의해 왔기 때문이다.
- 방식은 문자에 따라 1~4 byte로 표현된다. 
- 영문자(7-bit ASCII)는 그대로 1 byte로 표현된다. 
- 8-bit ASCII(>=128)나 한글 등 다국어 문자는 문자 당 2~4 byte로 변환된다.

### `bytes` 
`bytes`: immutable sequences of bytes
  - 문자열(str)이 아니다. 내용이 무엇이냐에 상관없이 8-bit인 byte들의 열이다.
  - str method 거의 전부 적용 가능

In [1]:
a = 'hello, 안녕'
b = a.encode('utf-8')
print(b)
print(b.hex())
print(len(b))  # number of bytes
print(type(b))

b'hello, \xec\x95\x88\xeb\x85\x95'
68656c6c6f2c20ec9588eb8595
13
<class 'bytes'>


### `bytearray` 
Mutable counterpart to `bytes` objects

In [2]:
ba = bytearray(a, encoding='utf-8')
# ba = bytearray(b)
ba.extend(b'korea')
ba[0] += 1
print(ba)

bytearray(b'iello, \xec\x95\x88\xeb\x85\x95korea')


In [3]:
ACK = 2
ptype, seq, checksum = 2, 5, 0x9a8b
header = bytearray([ptype, seq, checksum >> 8, checksum & 0xff])
print(header)

bytearray(b'\x02\x05\x9a\x8b')


## First UDP socket program - a client

In [4]:
import socket

# create an UDP socket (open UDP socket)
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 

# send message to the UDP echo server
s.sendto(b'Hello, World!', ('np.hufs.ac.kr', 7))   

# Wait for something arrival with buffer size == 2048 bytes
# and receive message and find the sender
message, sender = s.recvfrom(2048)           
print(sender)
print(message)

# close the socket
s.close()

('203.253.70.30', 7)
b'Hello, World!'


UDP는 connectionless protocol이므로  보낼(send) 때마다 목적지(to)를 지정한다.

받기 전에는 누가 보냈는지 모른다. 메시지가 도착해서 받을(recv) 때 비로소 누가 보냈는지(from) 알 수 있다.
- 메지지는 패킷 덩어리로 전달된다. 3차례에 걸쳐 메시지를 `sendto` 했으면, 상대방은 처음 `recvfrom`으로 첫번 째 message를 받고, 다음 `recvfrom`으로 두번 째 message를 받고, ...
- 다만, 어떤 메시지는 중간에 lost될 수 있다.
- 또한, 메시지가 너무(?) 크다면 짤릴 수 있다. 받아 보니, 일부만 도착할 수도...

> UDP socket을 이용하여 application에서 error없이 loss없이 신뢰성있게 주고 받으려면 UDP를 사용하지 말라.

> 그럼 언제??? 특별한 경우... 
> 약간이 손실은 참을 수 있는 multimedia application 등에서. (DNS protocol은 UDP socket으로 구현되어 있다.)

## First TCP socket program - a client
#### Open a TCP socket:

In [5]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # create a TCP socket object
print(s)
print(type(s))

<socket.socket fd=1072, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0>
<class 'socket.socket'>


#### Connect to the echo server:
- connection이 성공하면, connected socket은 local address와 remote(or foreign) address를 기억한다. (여기서 address는 *socket address*로 (IP address, port number)로 구성된다.
- 상대 쪽, 즉, server 쪽에서도 client의 socket address를 기억하고 있다.

In [6]:
s.connect(('np.hufs.ac.kr', 7))    # connect to echo server
print(s)

<socket.socket fd=1072, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('118.34.152.124', 58436), raddr=('203.253.70.30', 7)>


#### Send request message via the connected socket

In [7]:
msg = 'Hello, 대한민국!'
s.send(msg.encode('utf-8'))        # send in bytes or bytearray type

20

> 주의: 보내는 메시지는 UTF-8 code로 encoding, 받은 메시지는 UNICODE로 decoding
- TCP는 byte-stream protocol이다. 해야 한다. (UTF-8은 multi-byte code를 byte stream으로 *serialize*)
- Unicode와 같은 multi-byte code를 직접 보내면 위험하다.  
- 인터넷 대다수 protocol의 문법은 원래 ASCII로 사용을 전제로 개발되었고, web에서도 UTF-8가 표준

#### Receive reply message

In [8]:
reply = s.recv(1024)
print(type(reply))
print(reply.decode('utf-8'))       

<class 'bytes'>
Hello, 대한민국!


#### Close the socket:
- cause to send FIN (request to close the TCP session)
- no more use this socket

In [9]:
s.close()